In [ ]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sci

import sklearn.ensemble as se
import sklearn.model_selection as ms
import sklearn.metrics as sm

import seaborn as sns
import math
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from sklearn.inspection import permutation_importance
from sklearn.inspection import partial_dependence
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import shap
shap.initjs()
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [3]:
#Import the preprocessed data
pd.set_option('display.max_columns',None)
new_df = pd.read_excel('Preprocessed_data.xlsx')
new_df = new_df.iloc[:,1:]
new_df.describe()

,Reference,Latitude,Longitude,Depth,Temperature,Salinity,Density,O2,AT,CT,pH,pCO2,NO3,PO4,SiOH4,N2O,d15n,d18O,d15,d15Nb,SP,d15NO3,d18O-NO3,NO2,d15NO2
count,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2143.000000,2026.000000,2121.000000,1928.000000,1928.000000,2139.000000,481.000000,481.000000,1119.000000,180.000000
mean,6.189921,2.484248,-41.510641,479.530756,11.137797,34.601602,26.036709,103.096521,2311.867419,2220.966661,7.748110,846.410084,27.192629,2.097281,40.252332,32.789765,7.719646,54.165031,18.657786,-3.017231,21.334488,12.371139,10.994172,0.371242,-14.234812
std,5.056160,29.932197,110.952190,630.777420,7.329099,0.624860,2.040484,97.653462,41.673010,91.326575,0.154306,314.775392,11.361120,0.770417,35.449941,27.513083,3.102976,13.510201,7.482752,4.216389,9.946216,4.961233,5.568101,1.007430,11.015585
min,1.000000,-68.990000,-179.890909,0.000000,-1.743020,27.434599,12.046557,-0.080000,1993.526319,1840.403859,7.501403,210.335424,-0.355636,0.006641,0.839811,1.307829,-11.039096,12.400000,-1.081315,-33.884285,-9.419766,3.057646,1.601334,-0.040000,-34.259794
25%,3.000000,-16.000000,-107.500500,60.400000,4.817320,34.530000,25.662895,11.907500,2296.479282,2196.660840,7.619858,544.554811,23.423590,1.760589,18.782238,13.536400,6.042990,45.688984,14.007500,-5.076187,14.930997,8.348434,6.822577,0.010000,-22.925794
50%,3.000000,2.500000,-102.997000,204.900000,10.563496,34.640000,26.553022,76.182137,2306.012564,2229.012397,7.708984,847.621553,28.816909,2.184625,29.695503,26.015600,7.360000,49.850000,18.205110,-2.990000,21.260000,11.580000,10.704900,0.020000,-17.435000
75%,10.000000,19.000000,-70.644440,696.889000,15.614000,34.840250,27.219050,194.218750,2321.304786,2274.615883,7.876074,1113.372751,33.402288,2.635932,48.246675,44.125107,8.905990,56.795092,21.916021,-0.840000,25.820000,15.676431,15.309712,0.195000,-3.092057
max,19.000000,74.800000,286.510093,5117.200000,29.480000,36.570000,27.860000,400.865801,2456.239480,2414.440943,8.146143,1364.524934,46.753753,3.500509,197.908077,226.057678,23.740000,116.990000,52.000000,29.130000,60.371722,30.849737,26.227689,7.533074,10.883948


In [ ]:
#Split the dataset by three
#SP
df_SP = pd.concat([new_df.iloc[:,1:17],new_df.iloc[:,21]],axis=1)
display(df_SP.columns)
df_SP = df_SP.dropna()
df_SP.index=range(len(df_SP.index))
display(df_SP.describe())

In [ ]:
#15N
df_15N = pd.concat([new_df.iloc[:,1:17],new_df.iloc[:,17]],axis=1)
df_15N = df_15N.dropna(subset=[df_15N.columns[-1]])
df_15N.index=range(len(df_15N.index))
display(df_15N.columns)
display(df_15N.describe())

In [ ]:
#18O
df_18O = pd.concat([new_df.iloc[:,1:17],new_df.iloc[:,18]],axis=1)
df_18O = df_18Ol.dropna(subset=[df_18O.columns[-1]])
df_18O.index=range(len(df_18O.index))
display(df_18O.columns)
display(df_18O.describe())

## Split the dataset and screening the algorithm

In [ ]:
#Split the dataset 
df_PPRIs = [df_SP,df_15N,df_18O]
trainx,trainy,testx,testy=[],[],[],[]
ppris = ["SP","15N","18O"]
x1 = df_PPRIs[0].iloc[:,1:-1]
x2 = df_PPRIs[1].iloc[:,1:-1]
x3 = df_PPRIs[2].iloc[:,1:-1]
x = [x1,x2,x3]
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    #Natural logarithmic conversion of the target
    y = df_PPRIs[i].iloc[:,-1]
    ran_seed = 42

    train_x, test_x, train_y, test_y = ms.train_test_split(x[i], y, test_size = 0.2, random_state = ran_seed, shuffle=True)
    display(train_x)
    display(train_y)
    trainx.append(train_x)
    trainy.append(train_y)
    testx.append(test_x)
    testy.append(test_y)

In [ ]:
#Model performance with default hyperparameters
#Ensemble learning models 
estimators = [se.RandomForestRegressor(random_state=ran_seed),
             se.GradientBoostingRegressor(random_state=ran_seed),
             xgb.XGBRegressor(random_state=ran_seed),
             lgb.LGBMRegressor(random_state=ran_seed),
             cb.CatBoostRegressor(random_state=ran_seed, verbose=False)]
name = ['RF','GBDT','XGB','LGB','CB']
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    model_results=[]
    index_name =[]
    for n in range(len(estimators)):
        model = estimators[n]
        model.fit(trainx[i], trainy[i])

        scores = ms.cross_validate(model, trainx[i], trainy[i], cv=5, scoring=['r2','neg_root_mean_squared_error'])
        pred_train_y = model.predict(trainx[i])
        train_rmse = math.sqrt(sm.mean_squared_error(trainy[i], pred_train_y))

        index_name.append('{} default'.format(name[n]))
        model_results.append(np.array([model.score(trainx[i], trainy[i]),scores.get('test_r2').mean(),
                                        train_rmse, -scores.get('test_neg_root_mean_squared_error').mean()]))
    total_model_results = pd.DataFrame(model_results, index=index_name, columns=['train_R2','cv_R2','train_rmse','cv_rmse'])
    display(total_model_results)

## CatBoost

In [ ]:
#After tunning the hyperparameters by BayesSearch, the optimal CB model for each PPRIs is developed 
#The developed model will be exported as a pickle named "Triplet.pickle" , "Singlet.pickle" or "Hydroxyl.pickle" for the further application
import pickle
best_cbr = [cb.CatBoostRegressor(random_state=ran_seed, verbose=False,iterations=300,rsm=0.3,subsample=0.8,learning_rate=0.25,
                                   random_strength=1),
           cb.CatBoostRegressor(random_state=ran_seed, verbose=False,iterations=320,rsm=0.5,learning_rate=0.2,subsample=0.8,
                                   random_strength=1),
           cb.CatBoostRegressor(random_state=42, verbose=False,iterations=360,learning_rate=0.18,rsm=0.4,subsample=0.8,random_strength=84)]
filenames = ["SP.pickle", "15N.pickle", "18O.pickle"]

for model, filename in zip(best_cbr, filenames):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

df_PPRIs = [df_triplet,df_singlet,df_hydroxyl]
ppris = ["SP","15N","18O"]
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    best_cbr[i].fit(trainx[i], trainy[i])
    pred_train_y = best_cbr[i].predict(trainx[i])
    pred_test_y = best_cbr[i].predict(testx[i])

    scores = ms.cross_validate(best_cbr[i], trainx[i], trainy[i], cv=5, scoring=['r2','neg_root_mean_squared_error'])
    test_r = best_cbr[i].score(testx[i], testy[i])
    train_r = best_cbr[i].score(trainx[i], trainy[i])

    train_rmse = math.sqrt(sm.mean_squared_error(trainy[i], pred_train_y))
    test_rmse = math.sqrt(sm.mean_squared_error(testy[i], pred_test_y))
    cv_R2 = scores.get('test_r2').mean()
    cv_RMSE = -scores.get('test_neg_root_mean_squared_error').mean()

    print('CatBoost for {}:'.format(ppris[i]))
    print(best_cbr[i].get_all_params())
    print('\n R2  Train: {:.4f}'.format(train_r), ' Test: {:.4f}'.format(test_r), 
          ' CV: {:.4f} (+/- {:.4f})'.format(cv_R2,scores.get('test_r2').std()))
    print('RMSE Train: {:.4f}'.format(train_rmse),' Test: {:.4f}'.format(test_rmse), 
          ' CV: {:.4f} (+/- {:.4f})\n'.format(cv_RMSE, scores.get('test_neg_root_mean_squared_error').std())) 
    with open(filenames[i], "wb") as file:
            pickle.dump(model, file)

In [13]:
with open('best_cbr_SP.pickle', 'wb') as f:
    pickle.dump(best_cbr[0], f)
with open('best_cbr_15N.pickle', 'wb') as f:
    pickle.dump(best_cbr[1], f)
with open('best_cbr_18O.pickle', 'wb') as f:
    pickle.dump(best_cbr[2], f)

In [ ]:
#Feature importance
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('Figure_4.pdf')
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    fi = best_cbr[i].feature_importances_
    s = pd.Series(fi, trainx[i].columns)
    s.sort_values().plot.barh(color='darkgreen',figsize=[4,6])
    plt.xlabel("Feature importance", size=10)
    plt.tick_params(labelsize=10)
 #   plt.show()
    plt.savefig(pdf, format='pdf')
    plt.clf()
pdf.close()

In [ ]:
#SHAP summary plot
from matplotlib.colors import LinearSegmentedColormap
import shap
import matplotlib.pyplot as plt
colors = ["seagreen","tab:orange"]
cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
writer = pd.ExcelWriter('shap_values.xlsx',engine='openpyxl')
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    explainer = shap.TreeExplainer(best_cbr[i])
    shap_values = explainer(trainx[i]) 
    shap.summary_plot(shap_values, trainx[i],show = False, feature_names=trainx[i].columns, cmap=cmap1,alpha=0.8, plot_size=(5,6))
    plt.xlabel('SHAP value (model for ln{})'.format(ppris[i]),size=14)
    plt.show()
    shap_arr = np.array(shap_values)
    shap_df = pd.DataFrame(shap_arr, columns=trainx[i].columns)
    shap_df.to_excel(writer, sheet_name='Sheet{}'.format(i+1))
    plt.close()
#writer.save()
writer.close()

In [ ]:
#PDP plot for the five most important features
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
features1 = ['N2O','pH','O2','Latitude','Longitude','Density']
features2 = ['N2O','CT','Latitude','PO4','NO3','pCO2']
features3 = ['N2O','CT','O2','Latitude','pCO2','Depth']
features = [features1,features2,features3]
pdf = PdfPages('Figure_5_PDP_plot.pdf')
for i in range(3):    
    fig,ax = plt.subplots(figsize=(15,5))
    plt.title(label=[ppris[i]])
    #pdp = partial_dependence(best_cbr[i], trainx[i], features[i], kind='average', grid_resolution=10)
    #display = PartialDependenceDisplay([pdp], features=features[i],feature_names=[plt.title], target_idx=0, deciles=dict)
    #plt.show()
    #print('PDP:', pdp)
    #Resu= pdp.average[i].T
    display1 = PartialDependenceDisplay.from_estimator(best_cbr[i], trainx[i], features[i], kind='both', grid_resolution=100)
    display1.plot(ax=ax, n_cols=5)
    #plt.show()
    plt.savefig(pdf, format='pdf')
    plt.clf()
pdf.close()

In [ ]:
#pdf = PdfPages('Figure_5.pdf')
for i in range(3):
    print("______________{}______________".format([ppris[i]]))
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=trainy[i], y=best_cbr[i].predict(trainx[i]), s=100, color='tab:orange',  edgecolor='1',alpha=0.8)
    sns.scatterplot(x=testy[i], y=best_cbr[i].predict(testx[i]), s=100, color='darkgreen',  edgecolor='1',alpha=0.8)
    plt.legend(["Train", "Test"], loc='lower right', frameon=True, fontsize=18)
    sns.regplot(x=testy[i], y=best_cbr[i].predict(testx[i]), color='darkgreen',  scatter=False, truncate=False)
    sns.regplot(x=trainy[i], y=best_cbr[i].predict(trainx[i]), color='tab:orange', scatter=False, truncate=False)
    plt.tick_params(direction='out',length=4,width=0.8, labelsize=18, pad=5)

    plt.xlabel("Observed ln{}".format(ppris[i]), fontsize =20, fontname = 'Arial')
    plt.ylabel("Predicted ln{}".format(ppris[i]), fontsize =20, fontname = 'Arial')
   # plt.show()
    
##数据保存成csv
results = []

for i in range(3):
    train_predicted = best_cbr[i].predict(trainx[i])
    test_predicted = best_cbr[i].predict(testx[i])
    
    df_train = pd.DataFrame({'Observed': trainy[i], 'Predicted': train_predicted})
    df_test = pd.DataFrame({'Observed': testy[i], 'Predicted': test_predicted})
    
    # Save train and test dataframes as CSV files
    df_train.to_csv('train_results_{}.csv'.format(i), index=False)
    df_test.to_csv('test_results_{}.csv'.format(i), index=False)
    
    results.append((df_train, df_test))